In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [9]:
data = pd.read_csv('/Users/amulya/Desktop/Capstone/DSCI-601-Amy/Data/FeatureEngineered/AKAM_feature_engineeredv2.csv')

In [10]:
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

In [11]:
features = [['VOL_CHANGE', 'BA_SPREAD', 'ILLIQUIDITY', 'TURNOVER', 'MARKET_CAP', 'TRAN_COST', 'ASK', 'BID', 'day_of_week',
            'day_of_month', 'month', 'is_month_start', 'is_month_end', 'year', 'week', 'RET_ema_12', 'RET_ema_26',
            'RET_skew_10', 'RET_kurtosis_10', 'RET_entropy_10', 'RSI', 'OBV', 'RET_lag_1', 'RET_lag_7', 'RET_lag_14',
            'RET_lag_30', 'RET_mean_7d', 'RET_std_7d', 'RET_mean_14d', 'RET_std_14d', 'RET_mean_30d', 'RET_std_30d']]

targets = data[['PRC', 'RET']]

print(type(features), type(targets))

<class 'list'> <class 'pandas.core.frame.DataFrame'>


In [8]:
features.fillna(method='ffill', inplace=True)
targets.fillna(method='ffill', inplace=True)

AttributeError: 'list' object has no attribute 'fillna'

In [12]:

print(type(data))  # Should print <class 'pandas.core.frame.DataFrame'>

features = data[['VOL_CHANGE', 'BA_SPREAD', 'ILLIQUIDITY', 'sprtrn', 'TURNOVER', 
                 'MARKET_CAP', 'TRAN_COST', 'ASK', 'BID', 'RSI', 'OBV', 
                 'RET_ema_12', 'RET_ema_26', 'RET_lag_1', 'RET_lag_7', 
                 'RET_mean_7d', 'RET_std_7d']].copy()

# Ensure targets is a DataFrame
targets = data[['PRC', 'RET']].copy()

# Check types again
print(type(features), type(targets))  # Should both print <class 'pandas.core.frame.DataFrame'>

# Now handle missing values
features.fillna(method='ffill', inplace=True)
targets.fillna(method='ffill', inplace=True)

# Verify if there are still missing values
print(features.isnull().sum())
print(targets.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
VOL_CHANGE      0
BA_SPREAD       0
ILLIQUIDITY     0
sprtrn          0
TURNOVER        0
MARKET_CAP      0
TRAN_COST       0
ASK             0
BID             0
RSI            13
OBV             1
RET_ema_12      0
RET_ema_26      0
RET_lag_1       1
RET_lag_7       7
RET_mean_7d     7
RET_std_7d      8
dtype: int64
PRC    0
RET    0
dtype: int64


/var/folders/69/00tjyfg95_d37pp6ql47yf9h0000gn/T/ipykernel_80503/1774995446.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features.fillna(method='ffill', inplace=True)
/var/folders/69/00tjyfg95_d37pp6ql47yf9h0000gn/T/ipykernel_80503/1774995446.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  targets.fillna(method='ffill', inplace=True)


In [13]:

print(type(data))  
print(data.head()) 


<class 'pandas.core.frame.DataFrame'>
           TICKER     CUSIP                   COMNAM       RET  VOL_CHANGE  \
date                                                                         
1999-11-01   AKAM  00971T10  AKAMAI TECHNOLOGIES INC  0.200603    0.027675   
1999-11-02   AKAM  00971T10  AKAMAI TECHNOLOGIES INC  0.105773    0.061926   
1999-11-03   AKAM  00971T10  AKAMAI TECHNOLOGIES INC -0.009728    0.021790   
1999-11-04   AKAM  00971T10  AKAMAI TECHNOLOGIES INC -0.092338    0.087867   
1999-11-05   AKAM  00971T10  AKAMAI TECHNOLOGIES INC  0.005772    0.031662   

            BA_SPREAD   ILLIQUIDITY    sprtrn   TURNOVER       PRC  ...  \
date                                                                ...   
1999-11-01   0.000717  2.221163e-10 -0.006464  56.660812  0.531090  ...   
1999-11-02   0.000649  1.361950e-10 -0.004712  44.062958  0.587481  ...   
1999-11-03   0.000655 -3.256651e-11  0.005335  17.114236  0.581747  ...   
1999-11-04   0.000361 -3.308900e-10  0.0

In [9]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(data[features])

In [11]:
def create_sequences(data, target, time_steps=10):
    X, y = [], []
    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i, :])  # Select the last `time_steps` for each sequence
        y.append(target[i])  # Select the target value at the corresponding time step
    return np.array(X), np.array(y)

In [12]:
# for predicting `RET`
time_steps = 10
X_ret, y_ret = create_sequences(scaled_features, data[target_ret].values, time_steps)

# for `RET`
X_train_ret, X_test_ret, y_train_ret, y_test_ret = train_test_split(X_ret, y_ret, test_size=0.2, random_state=42)

# predicting `PRC`
X_prc, y_prc = create_sequences(scaled_features, data[target_prc].values, time_steps)

# for `PRC`
X_train_prc, X_test_prc, y_train_prc, y_test_prc = train_test_split(X_prc, y_prc, test_size=0.2, random_state=42)


In [13]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))  # Output layer with a single unit for regression
    model.compile(optimizer='adam', loss='mse')  # Compile the model with Mean Squared Error loss
    return model

In [14]:
#train for RET
print("Training LSTM model for predicting `RET`...")
model_ret = build_lstm_model((X_train_ret.shape[1], X_train_ret.shape[2]))
history_ret = model_ret.fit(X_train_ret, y_train_ret, epochs=50, batch_size=32, validation_split=0.2, verbose=1)
y_pred_ret = model_ret.predict(X_test_ret)

Training LSTM model for predicting `RET`...


/Users/amulya/Desktop/Capstone/DSCI-601-Amy/.venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: object __array__ method not producing an array

In [16]:
print("Training LSTM model for predicting `PRC`...")
model_prc = build_lstm_model((X_train_prc.shape[1], X_train_prc.shape[2]))
history_prc = model_prc.fit(X_train_prc, y_train_prc, epochs=50, batch_size=32, validation_split=0.2, verbose=1)
y_pred_prc = model_prc.predict(X_test_prc)

Training LSTM model for predicting `PRC`...


ValueError: object __array__ method not producing an array